In [ ]:
# This processes output csv files from ArcGIS into files that are injested into the pipeline.
# Specifically, read in ArcGIS csv files, extract relevant coordinates, and them write out to
#     another, 'processed' csv with additional info in a header

In [2]:
import ephem
import cartopy.crs as ccrs
import skyfield
import pandas as pd
import numpy as np
import ipdb
import matplotlib.pyplot as plt

%matplotlib qt

In [3]:
# read in raw file as written out by ArcGIS
df = pd.read_csv("./data/01_raw_gis_output/Export_Output_DHEKELIAWOENCLAVES_equid_164.txt")

In [4]:
df

,OBJECTID,Join_Count,TARGET_FID,FID_dhekelia_polygon_SymDiff1,FID_dhekelia_polygon,FID_ormidhia_enclave_polygon,FID_xylotymbo_enclave_polygon,LON,LAT,X_COORD,Y_COORD
0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,2,NaN,NaN,NaN,NaN,33.673992,34.943493,3.748572e+06,3.634062e+06
2,3,0,3,NaN,NaN,NaN,NaN,33.675466,34.943493,3.748736e+06,3.634062e+06
3,4,0,4,NaN,NaN,NaN,NaN,33.676939,34.943493,3.748900e+06,3.634062e+06
4,5,0,5,NaN,NaN,NaN,NaN,33.678412,34.943493,3.749064e+06,3.634062e+06
...,...,...,...,...,...,...,...,...,...,...,...
16862,16863,0,16863,NaN,NaN,NaN,NaN,33.911183,35.123831,3.774976e+06,3.650462e+06
16863,16864,0,16864,NaN,NaN,NaN,NaN,33.912657,35.123831,3.775140e+06,3.650462e+06
16864,16865,0,16865,NaN,NaN,NaN,NaN,33.914130,35.123831,3.775304e+06,3.650462e+06
16865,16866,0,16866,NaN,NaN,NaN,NaN,33.915603,35.123831,3.775468e+06,3.650462e+06


In [5]:
# apply cuts

preserve_df_1 = df.where(df["Join_Count"] == 1).dropna(how="all")
#preserve_df_2 = preserve_df_1.where(preserve_df_1["LAT"] < -60.).dropna(how="all")

#preserve_df_final = preserve_df_2.where(np.logical_and(preserve_df_1["LON"] < -20.,preserve_df_1["LON"] > -80.)).dropna(how="all")
preserve_df_final = preserve_df_1

In [5]:
plt.hist(preserve_df_final["LON"])
plt.show()

In [6]:
preserve_df_final

,OBJECTID,Join_Count,TARGET_FID,FID_dhekelia_polygon_SymDiff1,FID_dhekelia_polygon,FID_ormidhia_enclave_polygon,FID_xylotymbo_enclave_polygon,LON,LAT,X_COORD,Y_COORD
125,126.0,1.0,126.0,1.0,1.0,-1.0,-1.0,33.856674,34.943493,3.768908e+06,3.634062e+06
126,127.0,1.0,127.0,1.0,1.0,-1.0,-1.0,33.858147,34.943493,3.769072e+06,3.634062e+06
127,128.0,1.0,128.0,1.0,1.0,-1.0,-1.0,33.859620,34.943493,3.769236e+06,3.634062e+06
128,129.0,1.0,129.0,1.0,1.0,-1.0,-1.0,33.861093,34.943493,3.769400e+06,3.634062e+06
129,130.0,1.0,130.0,1.0,1.0,-1.0,-1.0,33.862567,34.943493,3.769564e+06,3.634062e+06
...,...,...,...,...,...,...,...,...,...,...,...
16672,16673.0,1.0,16673.0,1.0,1.0,-1.0,-1.0,33.877299,35.122025,3.771204e+06,3.650298e+06
16673,16674.0,1.0,16674.0,1.0,1.0,-1.0,-1.0,33.878772,35.122025,3.771368e+06,3.650298e+06
16674,16675.0,1.0,16675.0,1.0,1.0,-1.0,-1.0,33.880245,35.122025,3.771532e+06,3.650298e+06
16675,16676.0,1.0,16676.0,1.0,1.0,-1.0,-1.0,33.881719,35.122025,3.771696e+06,3.650298e+06


In [9]:
plt.clf()

ax = plt.axes(projection=ccrs.PlateCarree())
#ax = plt.axes(projection=ccrs.RotatedPole())

ax.scatter(preserve_df_final["LON"], preserve_df_final["LAT"], color="red", s=2, transform=ccrs.PlateCarree())
#ax.scatter(preserve_df_final["LON"], preserve_df_final["LAT"], color="red", transform=ccrs.RotatedPole())

ax.coastlines()
ax.set_global()

# Save the plot by calling plt.savefig() BEFORE plt.show()
#plt.savefig('coastlines.pdf')
#plt.savefig('coastlines.png')

plt.show()

In [29]:
# write out coordinates file with a header

technical_string = "brit_antarctica"
processed_file_name = technical_string+".csv"
hdr = open(processed_file_name, "w")

hdr.write('KEY;VALUE;COMMENT;REF\n')
hdr.write('technical_string;'+technical_string+';;\n')
hdr.write('display_string;British Antarctic Territory;;\n')
hdr.write('start_date;1962 March 3;Statutory Instrument - the British Antarctic Territory Order in Council 1962/400;britishantarcticterritory.org.uk\n')
hdr.write('end_date;-99999;;\n')
hdr.write('polygon_spacing;-99999;;\n')
hdr.write('other;;;\n')

hdr.write('##############################\n')

preserve_df_final.to_csv(hdr, columns=["LAT","LON"], index=False)
print("Wrote out " + processed_file_name)

Wrote out brit_antarctica.csv
